In [7]:
# Import libraries
import pandas as pd
import os
parent_dir = os.path.dirname(os.getcwd())
processed_dir = os.path.join(parent_dir, "processed_data")

citi_df = pd.read_csv(os.path.join(processed_dir, 'citi_bike_raw.csv'))
citi_df.head()

,tripduration,starttime,stoptime,start station id,start station name,start station latitude,start station longitude,end station id,end station name,end station latitude,end station longitude,bikeid,usertype,birth year,gender
0,789,2020-01-01 00:00:55.3900,2020-01-01 00:14:05.1470,504,1 Ave & E 16 St,40.732219,-73.981656,307,Canal St & Rutgers St,40.714275,-73.989900,30326,Subscriber,1992,1
1,1541,2020-01-01 00:01:08.1020,2020-01-01 00:26:49.1780,3423,West Drive & Prospect Park West,40.661063,-73.979453,3300,Prospect Park West & 8 St,40.665147,-73.976376,17105,Customer,1969,1
2,1464,2020-01-01 00:01:42.1400,2020-01-01 00:26:07.0110,3687,E 33 St & 1 Ave,40.743227,-73.974498,259,South St & Whitehall St,40.701221,-74.012342,40177,Subscriber,1963,1
3,592,2020-01-01 00:01:45.5610,2020-01-01 00:11:38.1550,346,Bank St & Hudson St,40.736529,-74.006180,490,8 Ave & W 33 St,40.751551,-73.993934,27690,Subscriber,1980,1
4,702,2020-01-01 00:01:45.7880,2020-01-01 00:13:28.2400,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014,3637,Fulton St & Waverly Ave,40.683239,-73.965996,32583,Subscriber,1982,1


In [8]:
# Feature Engineering
citi_df['starttime'] = pd.to_datetime(citi_df['starttime'])
citi_df['stoptime'] = pd.to_datetime(citi_df['stoptime'])
citi_df['od_pair'] = list(zip(citi_df['start station id'], citi_df['end station id']))
citi_df['age'] = citi_df['starttime'].dt.year - citi_df['birth year']
# Extract Time Features
citi_df['start_hour'] = citi_df['starttime'].dt.hour
citi_df['start_day_of_week'] = citi_df['starttime'].dt.day_of_week
# Creating Station df & Drop unnecessary columns
start_station_df = citi_df[['start station id', 'start station name', 'start station latitude','start station longitude']].drop_duplicates()
start_station_df.rename(columns ={'start station id':'station_id', 'start station name': 'station_name', 'start station latitude':'latitude','start station longitude':'longitude'}, inplace=True)
end_station_df = citi_df[['end station id', 'end station name', 'end station latitude','end station longitude']].drop_duplicates()
end_station_df.rename(columns ={'end station id':'station_id', 'end station name': 'station_name', 'end station latitude':'latitude','end station longitude':'longitude'}, inplace=True)
stations_df = pd.concat([start_station_df, end_station_df], axis=0).reset_index(drop=True)
# Clean up citi bike
citi_df = citi_df.drop(['start station name', 'start station latitude','start station longitude','end station name', 'end station latitude','end station longitude'], axis=1)
citi_df = citi_df.drop(['end station id','start station id'], axis=1)
citi_df = citi_df.drop(['birth year'], axis=1)
# Renaming columns
citi_df = citi_df.rename(columns={'tripduration':'trip_duration', 'starttime':'start_time','stoptime':'stop_time', 'bikeid':'bike_id','usertype':'user_type'})

In [9]:
citi_df = citi_df[['od_pair', 'bike_id', 'user_type', 'gender', 'age', 'trip_duration', 'start_time', 'start_hour', 'start_day_of_week']]
citi_df.head()

,od_pair,bike_id,user_type,gender,age,trip_duration,start_time,start_hour,start_day_of_week
0,"(504, 307)",30326,Subscriber,1,28,789,2020-01-01 00:00:55.390,0,2
1,"(3423, 3300)",17105,Customer,1,51,1541,2020-01-01 00:01:08.102,0,2
2,"(3687, 259)",40177,Subscriber,1,57,1464,2020-01-01 00:01:42.140,0,2
3,"(346, 490)",27690,Subscriber,1,40,592,2020-01-01 00:01:45.561,0,2
4,"(372, 3637)",32583,Subscriber,1,38,702,2020-01-01 00:01:45.788,0,2


In [10]:
stations_df.head()

,station_id,station_name,latitude,longitude
0,504,1 Ave & E 16 St,40.732219,-73.981656
1,3423,West Drive & Prospect Park West,40.661063,-73.979453
2,3687,E 33 St & 1 Ave,40.743227,-73.974498
3,346,Bank St & Hudson St,40.736529,-74.006180
4,372,Franklin Ave & Myrtle Ave,40.694546,-73.958014


In [12]:
stations_df.to_csv(os.path.join(processed_dir, 'stations.csv'), index=False)
citi_df.to_csv(os.path.join(processed_dir, 'citi_bike_processed.csv'), index=False)

In [13]:
citi_df.shape, stations_df.shape

((1240596, 9), (1901, 4))

In [14]:
citi_df.groupby('gender')['age'].agg(count='count').reset_index()

,gender,count
0,0,56968
1,1,904522
2,2,279106


In [15]:
citi_df.groupby('user_type')['age'].agg(count='count').reset_index()

,user_type,count
0,Customer,94777
1,Subscriber,1145819
